In [5]:
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool
from pydantic_settings import BaseSettings
# from file_path import file_path
from pathlib import Path

class Settings(BaseSettings):
    GOOGLE_API_KEY:str

setting = Settings()

os.environ["GOOGLE_API_KEY"] = setting.GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
# path = file_path(input("Enter or paste path of pdf file you want to load to llm's memory"))]
desktop = os.path.join(os.path.expanduser("~"), "Desktop")      # → /Users/you/Desktop
file_name = "africa_econ.pdf"                                      # change this
file_path = os.path.join(desktop, file_name)

try:
    if not os.path.isfile(file_path):
        raise FileNotFoundError("The path does not exist")
    
    file_pages = PyPDFLoader(file_path).load()
except Exception as e:
    print(str(e))
    raise

In [21]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=120)
docs = splitter.split_documents(file_pages)

In [23]:
chroma_dir = "./Africonomy"

if not os.path.isdir(chroma_dir):
    os.mkdir(chroma_dir)

try:
        vector_store = Chroma.from_documents(
        documents=docs,
        persist_directory=chroma_dir,
        embedding=embeddings,
        collection_name="Africonomy"
    )
except Exception as e:
      print(str(e))
      raise

In [25]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":80, "score_threshold":0.5}
)

In [26]:
@tool
def vector_search(query:str)-> str:
    """Searches the vector database to and retrives documents based on recieved query."""

    docs = retriever.invoke(query)

    if not docs:
        return "No document matches the query in the vector db"
    
    response: list[str] =[]
    
    for i, doc in enumerate(docs):
        response.append(f"document {i+1}: {doc}")

    return "\n ".join(response)


In [ ]:
tools = [vector_search]
llm = llm.bind_tools(tools)

class AgenState(TypedDict):
    messages:Annotated[Sequence[BaseMessage], add_messages]

tools_dict = {tool.name:tool for tool in tools}

sys_prompt = ""

In [30]:
def call_llm(state:AgenState)-> AgenState:
    """Calls the llm with state messages and system prompt and returns response of the llm"""

    messages = list(state['messages']) + [SystemMessage(content=sys_prompt)]

    response = llm.invoke(messages)

    return {"messages":[response]}

In [31]:
def should_continue(state:AgenState)->bool:
    """Checks the last message for tool calls to determine if to call another tool or end the conversation."""

    last_msg = state['messages'][-1]

    return hasattr(last_msg, "tool_calls") and len(last_msg.tool_calls) > 0

In [32]:
# def invoke_tool(state:AgenState)-> AgenState:
#     """Invokes the needed tool and checks that the tool name exists."""

#     tool_calls = state['messages'][-1].tool_calls